# NN Model II
- User tf.keras
- consider only numerical data, no levels

In [50]:
import tensorflow
from tensorflow import keras
import tensorflow.keras.layers as layers

import sys
import numpy as np
import os
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

from kedro.extras.datasets.pandas import CSVDataSet

### Loading Data

In [9]:
BattleResults = catalog.load('Battle_Results_Train')

2020-05-24 18:23:20,748 - kedro.io.data_catalog - INFO - Loading data from `Battle_Results_Train` (CSVDataSet)...


In [10]:
BattleResults.head()

,Name_1,Level_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult
0,Psyduck,8,68,65,59,82,63,69,False,Weedle,75,336,192,172,103,103,279,False,Unknown,-336
1,Rapidash,86,691,707,492,571,571,752,False,Zapdos,72,607,523,499,722,523,577,True,Sunshine,133
2,Dratini,93,636,698,484,538,538,538,False,Marowak,73,446,529,720,328,529,284,False,Windy,324
3,Horsea,6,41,47,85,85,29,72,False,Moltres,86,919,886,797,1088,750,797,True,Night,-919
4,Golbat,78,571,519,449,400,481,567,False,Kingler,96,637,1178,1049,456,456,682,False,Unknown,-560


In [136]:
len(BattleResults.loc[BattleResults["Legendary_1"]==True])

50666

### Data Pipeline

In [11]:
def RelativeResult(X):
    HP = X.apply(lambda row: row["HP_1"] if row["BattleResult"] > 0 else row["HP_2"], axis=1).values
    relativized = X["BattleResult"].values / HP
    return relativized

y = RelativeResult(BattleResults)

In [46]:
BattleResults_train, BattleResults_val, y_train, y_val  = train_test_split(BattleResults, y, test_size=0.2, random_state=42)

In [51]:
numerical_attributes = ['HP_1', 'Attack_1', 'Defense_1', 'Sp_Atk_1', 'Sp_Def_1', 'Speed_1', 
                          'HP_2', 'Attack_2', 'Defense_2', 'Sp_Atk_2', 'Sp_Def_2', 'Speed_2']
full_pipeline = ColumnTransformer([
        ("Numerical Attributes", StandardScaler(), numerical_attributes)
    ])
full_pipeline.fit(BattleResults); #battle_data

In [52]:
x_train = full_pipeline.transform(BattleResults_train)
x_val = full_pipeline.transform(BattleResults_val)

In [85]:
x_train_small,_, y_train_small, _  = train_test_split(x_train, y_train, train_size=0.5, random_state=42)

### Model

In [152]:
# 10 epochs, Validation  RMSE: 0.13
model = keras.models.Sequential([
    layers.Input(shape=(x.shape[1],)),
    layers.Dense(200, activation='relu'),
    layers.Dense(200, activation='relu'),
    layers.Dense(200, activation='relu'),
    layers.Dense(1, activation='tanh')
])

In [148]:
# 60 epochs, Validation  RMSE: 0.15
model = keras.models.Sequential([
    layers.Input(shape=(x.shape[1],)),
    layers.Dense(400, activation='relu'),
    layers.Dense(200, activation='relu'),
    layers.Dense(1, activation='tanh')
])

In [144]:
# 30 epochs, Validation  RMSE: 0.13
model = keras.models.Sequential([
    layers.Input(shape=(x.shape[1],)),
    layers.Dense(800, activation='relu'),
    layers.Dense(400, activation='relu'),
    layers.Dense(400, activation='relu'),
    layers.Dense(400, activation='relu'),
    layers.Dense(400, activation='relu'),
    layers.Dense(400, activation='relu'),
    layers.Dense(1, activation='tanh')
])

In [153]:
model.compile(loss='mse',
              optimizer=keras.optimizers.RMSprop(learning_rate=3e-3),
              metrics=['RootMeanSquaredError'])

Models are not really overfitting. Time to add aditional data.

### Train

In [ ]:
history = model.fit(x_train, y_train, epochs=60,
                    validation_data=(x_val, y_val),
                    batch_size=1024)